# Сбор и разметка данных для машинного обучения

## Домашняя работа №1. Датасет и веб-скрейпинг.

### Defenition of done
<ul style='font-size: 12pt'>
    <li>Соберите данные из двух разных источников (открытый датасет + веб-скрейпинг или API).
        <p>
        Стоит учитывать, что датасет, который вы сформируете на данном этапе пригодится вам для дальнейшего задания.</p>
    </li>
    <li>Проведите их агрегацию, создав единый датасет. </li>
    <li> Проведите разведывательный анализ данных (EDA). </li>
    <li> Постройте базовые визуализации для основных признаков с учетом разметки данных. </li>
    <li> Опишите возможные применения этих данных в контексте машинного обучения.</li>
</ul>

In [1]:
# установка ffmpeg
! choco install ffmpeg-full

# установка библиотек
! pip install numpy pandas matplotlib bs4 requests pytube

In [1]:
# подключение библиотек
import os
import time
import random
import requests
import re
import json

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from pytube import YouTube

In [76]:
# получить список url-видео из плейлиста
def scrape_videos(url):
    req = requests.get(url)
    send = BeautifulSoup(req.text, "html.parser") 
    search = send.find_all("script")
    key = '"videoId":"'
    data = re.findall(key + r"([^*]{11})", str(search))
    
    return data

# скачать видео с YouTube по ссылке
def download(link, vid):
    youtubeObject = YouTube(link)    
    try:
        youtubeObject = youtubeObject.streams.get_highest_resolution()
        youtubeObject.download(f".\\data\\videos\\", filename=f'video_{vid}.mp4')
        print("Download is completed successfully")        
    except:
        print("An error has occurred")

## 1. Датасет 'Russian traffic sign images dataset'

Чтение json-файлов

In [77]:
f_train = open('./data/rtsd-dataset/train_anno.json')
f_val = open('./data/rtsd-dataset/val_anno.json')
 
data_train = json.load(f_train)
data_val = json.load(f_val)
  
f_train.close()
f_val.close()

Формирование DataFrame

In [78]:
df_train_img = pd.DataFrame(data_train['images'])
df_train_annot = pd.DataFrame(data_train['annotations'])
df_train_cat = pd.DataFrame(data_train['categories'])

df_val_img = pd.DataFrame(data_val['images'])
df_val_annot = pd.DataFrame(data_val['annotations'])
df_val_cat = pd.DataFrame(data_val['categories'])

INNER JOIN таблиц 'images', 'annotations' и 'categories'

In [79]:
df_train = pd.merge(
    df_train_img,
    df_train_annot,
    how="inner",
    left_on="id",
    right_on="image_id"
)

df_train = pd.merge(
    df_train,
    df_train_cat,
    how="inner",
    left_on="category_id",
    right_on="id"
)

df_train.drop(columns=['id_x', 'id_y', 'id'], inplace=True)

df_train.head(5)

,width,height,file_name,image_id,category_id,area,bbox,iscrowd,name
0,1280,720,rtsd-frames/autosave01_02_2012_09_13_33.jpg,0,1,324,"[649, 376, 18, 18]",0,2_1
1,1280,720,rtsd-frames/autosave01_02_2012_09_13_34.jpg,1,1,420,"[671, 356, 20, 21]",0,2_1
2,1280,720,rtsd-frames/autosave01_02_2012_09_13_35.jpg,2,1,702,"[711, 332, 27, 26]",0,2_1
3,1280,720,rtsd-frames/autosave01_02_2012_09_13_36.jpg,3,1,1332,"[764, 290, 37, 36]",0,2_1
4,1280,720,rtsd-frames/autosave01_02_2012_09_13_37.jpg,4,1,3192,"[876, 200, 56, 57]",0,2_1


In [80]:
df_val = pd.merge(
    df_val_img,
    df_val_annot,
    how="inner",
    left_on="id",
    right_on="image_id"
)

df_val = pd.merge(
    df_val,
    df_val_cat,
    how="inner",
    left_on="category_id",
    right_on="id"
)

df_val.drop(columns=['id_x', 'id_y', 'id'], inplace=True)

df_val.head(5)

,width,height,file_name,image_id,category_id,area,bbox,iscrowd,name
0,1280,720,rtsd-frames/autosave10_10_2012_13_50_36_1.jpg,22978,3,810,"[622, 375, 30, 27]",0,1_17
1,1920,1080,rtsd-frames/autosave16_04_2013_15_11_26_1.jpg,34704,3,667,"[1050, 436, 29, 23]",0,1_17
2,1920,1080,rtsd-frames/autosave16_04_2013_13_19_50_2.jpg,32165,3,1720,"[1033, 506, 43, 40]",0,1_17
3,1920,1080,rtsd-frames/autosave13_04_2013_11_07_29_0.jpg,25166,3,1260,"[1185, 380, 36, 35]",0,1_17
4,1920,1080,rtsd-frames/autosave16_04_2013_13_19_50_0.jpg,32163,3,980,"[979, 530, 35, 28]",0,1_17


Тренировочную и валидационную выборки объединим

In [81]:
df = pd.concat([df_train, df_val], axis=0)
df = df.rename(columns={'name': 'category_name'})

df.head(5)

,width,height,file_name,image_id,category_id,area,bbox,iscrowd,category_name
0,1280,720,rtsd-frames/autosave01_02_2012_09_13_33.jpg,0,1,324,"[649, 376, 18, 18]",0,2_1
1,1280,720,rtsd-frames/autosave01_02_2012_09_13_34.jpg,1,1,420,"[671, 356, 20, 21]",0,2_1
2,1280,720,rtsd-frames/autosave01_02_2012_09_13_35.jpg,2,1,702,"[711, 332, 27, 26]",0,2_1
3,1280,720,rtsd-frames/autosave01_02_2012_09_13_36.jpg,3,1,1332,"[764, 290, 37, 36]",0,2_1
4,1280,720,rtsd-frames/autosave01_02_2012_09_13_37.jpg,4,1,3192,"[876, 200, 56, 57]",0,2_1


## 2. Веб-скрейпинг

Скачиваем видео с Youtube-плейлиста [Примеры видеозаписи с видеорегистраторов](https://www.youtube.com/playlist?list=PLqBt9etwHwFwRZ28oaW5c7gBd8McZ0YlV). \
Сохраняем в папку './data/videos/'

In [ ]:
video_path = './data/videos/'
img_path = './data/imgs/'
playlist_url = "https://www.youtube.com/playlist?list=PLqBt9etwHwFwRZ28oaW5c7gBd8McZ0YlV"

In [4]:
vid = scrape_videos(playlist_url)
vid = vid[::3]
vid = vid[:-1]

for i in vid:
    link = "https://www.youtube.com/watch?v=" + i
    download(link, i)
    time.sleep(2 + random.random())

Download is completed successfully
Download is completed successfully
An error has occurred
Download is completed successfully
Download is completed successfully
An error has occurred
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
An error has occurred
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
Download is completed successfully
An error has occurred
Download is completed successfully
Download is completed successfully
Download is completed successfully
Do

Причина возникающих ошибок при скачивании - возрастное ограничение, необходима авторизация. \
Установлено 44/50 видео

Делаем скриншот видео каждые 2 секунды.\
Сохраняем в папку './data/imgs/'

In [ ]:
video_names = os.listdir(video_path)
for i in range(len(video_names)):
    video_name = video_names[i]
    command = f"ffmpeg -i \"{video_path + video_name}\" -vf fps=1/2 {img_path + f'img{i + 1}_%03d.jpg'}"
    os.system(command)

Собрано 2 415 снимков.

## 3. Агрегация

<p style='font-size: 12pt'>
Агрегирование на данном этапе сбора данных не имеет смысла по нескольким причинам:
    <ul style='font-size: 12pt'>
        <li> в EDA будут участвовать только размеченные данные, это данные датасета;</li>
        <li> в таблице одна строка - это обнаруженный знак на изображении. Таким образом в датасете несколько строк ссылаются на одну и ту же фотографию. При добавлении к датасету данных, полученных в результате веб-скрейпинга (сырые данные), каждая новая строчка будет соответствовать только одной фотографии. В модуле 2 эта ситуация будет исправлена (разметка данных), и EDA дополнен. </li>
    </ul>
</p>
    
<p style='font-size: 12pt'>
Таким образом, на данном этапе сбора данных нет необходимости агрегировать размеченные данные и сырые данные, это лишняя задача, результаты которой не будут использованы в дальнейшем.
</p>

In [82]:
df.to_csv('./data/dataset_signs.csv', index=False)